The ***Olympics Data Analysis Project*** aims to delve into the rich history of the Olympic Games, from their revival in **1896** to the **2016** Games.
This project provides a detailed examination of over a century of data, uncovering trends, patterns, and insights that
 highlight the evolution of the Olympics over time.

***Beyound Medals:A data Driven look at olympics***


In [ ]:
import numpy as np
import pandas as pd

: 

## Importing Datasets

In [317]:
df = pd.read_csv('athlete_events.csv')
region_df = pd.read_csv('noc_regions.csv')

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:

df1 = df[df['Season'] == 'Summer']

In [ ]:
df1.shape

In [ ]:
region_df.tail()

## Merging both Datasets

In [324]:
df2 = df1.merge(region_df, on='NOC', how= 'left')   

In [ ]:
df2.tail()

In [ ]:
df2['region'].value_counts()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.duplicated().sum()

In [ ]:
df2.drop_duplicates(inplace=True)

In [ ]:
df2.duplicated().sum()

In [ ]:
df2['Medal'].value_counts()

In [ ]:
pd.get_dummies(df2['Medal']).head()

In [333]:
df3 = pd.concat([df2, pd.get_dummies(df2['Medal'])], axis = 1)

In [ ]:
df3.head(3)

In [ ]:
df3.groupby('NOC').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold', ascending=False).reset_index().head()

In [ ]:
medal_tally = df3.drop_duplicates(subset= ['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'])
medal_tally.head()

In [ ]:
medal_tally1 = medal_tally.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold', ascending=False).reset_index()
medal_tally1.head()

In [ ]:
medal_tally1['total'] = medal_tally1['Gold'] + medal_tally1['Silver'] + medal_tally1['Bronze'] 
medal_tally1.head()

In [ ]:
df3['Year'].unique()

In [340]:
year = df3['Year'].unique().tolist()
year.sort()

In [341]:
year.insert(0, 'Overall')      

In [ ]:
year

In [343]:
country = np.unique(df3['region'].dropna().values).tolist()
country.sort()

In [344]:
country.insert(0, 'Overall')

In [ ]:
country

## Medal Tally Function

In [347]:
def fetch_medal_tally(df, year, country):
    medal_df = df.drop_duplicates(subset= ['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'])
    flag = 0
    if year == 'Overall' and country == 'Overall':
        temp_df = medal_df
    if year == 'Overall' and country != 'Overall':
        flag = 1
        temp_df = medal_df[medal_df['region'] == country]
    if year != 'Overall' and country == 'Overall':
        temp_df = medal_df[medal_df['Year'] == int(year)]
    if year != 'Overall' and country != 'Overall':
        temp_df = medal_df[(medal_df['Year'] == int(year)) & (medal_df['region'] == country)]
    if flag == 1:
        temp_df = temp_df.groupby('Year').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Year').reset_index()
    else:
        temp_df = temp_df.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold', ascending=False).reset_index()
        
    temp_df['total'] = temp_df['Gold'] + temp_df['Silver'] + temp_df['Bronze'] 
    print(temp_df)
    

In [ ]:
medal_df = df3.drop_duplicates(subset= ['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'])
medal_df.head() 

In [ ]:

fetch_medal_tally(df3,'Overall', 'India')


### Medal Tally is complete

# Overall Analysis start

In [ ]:
# no. of editions
df3['Year'].unique().shape[0] -1

In [ ]:
# no. of cities
df3['City'].unique().shape

In [ ]:
# No. of sports
df3['Sport'].unique().shape

In [ ]:
# No. of events
df3['Event'].unique().shape

In [ ]:
# No. of athletes
df3['Name'].unique().shape

In [ ]:
# participating nations
df3['region'].unique().shape[0]

# Plotting

In [ ]:
nations_over_time = df3.drop_duplicates(['Year', 'region'])['Year'].value_counts().reset_index().sort_values('Year')
nations_over_time.head()

In [360]:
nations_over_time.rename(columns={'Year':'Edition', 'count':'No. of Countries'}, inplace= True)

In [361]:
import plotly.express as px

In [ ]:
fig = px.line(nations_over_time, x= 'Edition', y= 'No. of Countries')
fig.show()

In [ ]:
events_over_time = df3.drop_duplicates(['Year', 'Event'])['Year'].value_counts().reset_index().sort_values('Year')
events_over_time.rename(columns={'Year':'Edition', 'count':'No. of Events'}, inplace= True)
fig = px.line(events_over_time, x= 'Edition', y= 'No. of Events')
fig.show()

In [364]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(pd.pivot_table(df3, index='Sport', columns='Year', values='Event', aggfunc=pd.Series.nunique).fillna(0).astype('int'), annot = True)

In [366]:
x = df3.drop_duplicates(['Year','Event','Sport'])

## Successful Athletes in Olympics

In [368]:
def most_successful(df, sport):
    temp_df = df.dropna(subset=['Medal'])

    if sport != 'Overall':
        temp_df = temp_df[temp_df['Sport'] == sport]
    
    x = temp_df['Name'].value_counts().reset_index().head(15).merge(df3,left_on='Name',right_on='Name',how='left')[['Name','count','Sport','region']].drop_duplicates('Name')
    x.rename(columns={'count':'Medals'}, inplace=True)
    return x

In [ ]:
most_successful(df3, 'Overall').head()

# Country-wise Analysis

* Countrywise medal tally per year(line plot)
* What countries are good at heamap
* most successful athletes (Top 10)

In [372]:
# those who having nan medal no need
temp_df = df3.dropna(subset=['Medal'])
temp_df.drop_duplicates(subset=['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'], inplace=True)

In [373]:
new_df = temp_df[temp_df['region']=='USA']
final_df = new_df.groupby('Year').count()['Medal'].reset_index()

In [ ]:
fig = px.line(final_df, x= 'Year', y= 'Medal')
fig.show()

# Now we are creating a heatmap for seeing which country is good in which sport

In [ ]:
# For particular countrywise sport
new_df = temp_df[temp_df['region']=='USA']
plt.figure(figsize=(20,25))
sns.heatmap(new_df.pivot_table(index='Sport', columns='Year', values='Medal', aggfunc='count').fillna(0), annot=True)

In [ ]:
# For overall sport
pt = temp_df.pivot_table(index='Sport', columns='Year', values='Medal', fill_value=0, aggfunc='count')
plt.figure(figsize=(20,25))
sns.heatmap(pt, annot = True, fmt=".1f")

In [ ]:

region_grouped = temp_df.groupby('region').count().reset_index().head(134)
pd.set_option('display.max_rows', 134)
region_grouped.head(134)
# region_grouped[region_grouped['region'] == 'Albania']
# many regions dont have any value so by hardcore values i am removing them from list & passing only valid regions
region_list = region_grouped['region'].dropna().unique().tolist()
region_list

# Most Successful Players in a country

In [380]:
def most_successful(df, country):
    temp_df = df.dropna(subset=['Medal'])

    if country != 'Overall':
        temp_df = temp_df[temp_df['region'] == country]
    x = temp_df['Name'].value_counts().reset_index().head(10).merge(df, left_on='Name', right_on='Name', how='left')[
        ['Name', 'count', 'Sport']].drop_duplicates('Name')

    x.rename(columns={'count': 'Medals'}, inplace=True)
    return x

In [ ]:
most_successful(df3, 'Jamaica')

# Athlete-wise Analysis

In [ ]:
df3.head()

In [384]:
# we have duplicate athlete names since they played for multiple times
athlete_df = df3.drop_duplicates(subset= ['Name', 'region'])

In [ ]:
# dropping missing age na values
athlete_df['Age'].dropna()

In [386]:
import plotly.figure_factory as ff

## Age Distribution

In [388]:
x1 = athlete_df['Age'].dropna()
x2 = athlete_df[athlete_df['Medal']=='Gold']['Age'].dropna()
x3 = athlete_df[athlete_df['Medal']=='Silver']['Age'].dropna()
x4 = athlete_df[athlete_df['Medal']=='Bronze']['Age'].dropna()


In [ ]:
fig = ff.create_distplot([x1,x2,x3,x4], ['Overall Age','Gold Medalist','Silver Medalist','Bronze Medalist'], show_hist=False, show_rug=False)
fig.show()

In [ ]:
x = []
name = []
famous_sports = ['Basketball', 'Judo', 'Football', 'Tug-Of-War', 'Athletics',
                     'Swimming', 'Badminton', 'Sailing', 'Gymnastics',
                     'Art Competitions', 'Handball', 'Weightlifting', 'Wrestling',
                     'Water Polo', 'Hockey', 'Rowing', 'Fencing',
                     'Shooting', 'Boxing', 'Taekwondo', 'Cycling', 'Diving', 'Canoeing',
                     'Tennis', 'Golf', 'Softball', 'Archery',
                     'Volleyball', 'Synchronized Swimming', 'Table Tennis', 'Baseball',
                     'Rhythmic Gymnastics', 'Rugby Sevens',
                     'Beach Volleyball', 'Triathlon', 'Rugby', 'Polo',
                     'Ice Hockey']

for sport in famous_sports:
    temp_df = athlete_df[athlete_df['Sport'] == sport]
    x.append(temp_df[temp_df['Medal'] == 'Gold']['Age'].dropna())     
    name.append(sport)      

fig = ff.create_distplot(x, name, show_hist=False, show_rug=False)
fig.update_layout(autosize= False, width=1000, height=600)
fig.show()

## Height vs Weight plot

In [392]:
athlete_df['Medal'].fillna('No Medal', inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
temp_df = athlete_df[athlete_df['Sport'] == 'Athletics']

sns.scatterplot(x=temp_df['Weight'], y=temp_df['Height'], hue=temp_df['Medal'], style=temp_df['Sex'], s=100)

In [394]:
men = athlete_df[athlete_df['Sex'] == 'M'].groupby('Year').count()['Name'].reset_index()
women = athlete_df[athlete_df['Sex'] == 'F'].groupby('Year').count()['Name'].reset_index()

In [ ]:
final = men.merge(women, on= 'Year', how='left')
final.rename(columns={'Name_x':'Male', 'Name_y':'Female'}, inplace=True)
final.head()

In [ ]:
final.fillna(0, inplace=True)
final.head()

In [ ]:
fig = px.line(final, x= 'Year', y = ['Male', 'Female'])
fig.show()